In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import datetime
from datetime import datetime, timedelta
import json
import itertools
from tqdm import tqdm
import os
os.chdir("/content/drive/MyDrive/projects/Scrabble_Player_Rating/")
#import models
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [2]:
df_train = pd.read_csv("data/train_f.csv")
df_test = pd.read_csv("data/test_f.csv")
df_train.head()

,game_id,rating,nickname,score,turn_type_Play,turn_type_End,turn_type_Exchange,turn_type_Pass,turn_type_Timeout,turn_type_Challenge,...,cumm_turn_type_End_average,cumm_turn_type_Exchange_average,cumm_turn_type_None_average,cumm_turn_type_Pass_average,cumm_turn_type_Play_average,cumm_turn_type_Six-Zero Rule_average,cumm_turn_type_Timeout_average,difficult_letters_count_diff,difficult_words_count_diff,play_counts_diff
0,1,1500.0,1,429,13,1,0,0,0,0,...,0.371681,0.389381,0.000000,0.008850,13.318584,0.000000,0.053097,-8.0,-2.0,0
1,3,1811.0,2,440,13,1,0,0,0,0,...,0.409357,0.116959,0.005848,0.029240,14.023392,0.005848,0.029240,1.0,0.0,1
2,4,1473.0,3,119,12,0,2,0,0,0,...,0.277108,0.771084,0.000000,1.349398,11.481928,0.000000,0.156627,0.0,-3.0,253
3,5,1500.0,1,325,14,1,1,0,0,0,...,0.395556,0.426667,0.000000,0.040000,12.924444,0.000000,0.040000,4.0,-2.0,255
4,6,2029.0,4,378,12,0,0,0,0,0,...,0.475177,0.170213,0.000000,0.049645,12.312057,0.014184,0.000000,2.0,1.0,1


In [3]:
X_train = df_train.drop(["game_id", "nickname", "rating"], axis=1).to_numpy()
y_train = df_train["rating"].to_numpy()
X_test = df_test.drop(["game_id", "nickname"], axis=1).to_numpy()

In [4]:
# 設定ファイルの読み込み
conf = json.load(open("config/config.json"))

random_state = conf["random_state"]
n_splits = conf["n_splits"]

In [5]:
# GroupKFoldを使ってハイパラ探索

user_id = df_train["nickname"]
unique_user_ids = df_train["nickname"].unique()

train_scores = []
val_scores = []
# ユーザのnicknameリストをn_split分割し、それぞれ学習と検証として使う
kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

for tr_group_idx, va_group_idx in kf.split(unique_user_ids):
  # 学習に使うnicknameと検証に使うnicknameを抜き出す
  tr_groups, va_groups = unique_user_ids[tr_group_idx], unique_user_ids[va_group_idx]

  is_tr = user_id.isin(tr_groups) # 学習データのindex
  is_va = user_id.isin(va_groups) # 検証データのindex
  tr_x, va_x = X_train[is_tr], X_train[is_va]
  tr_y, va_y = y_train[is_tr], y_train[is_va]

  # 各Foldでの学習データでの精度を格納
  model = LinearRegression()
  model.fit(tr_x, tr_y)
  y_pred = model.predict(tr_x)
  train_scores.append(mean_squared_error(tr_y, y_pred, squared=False))
  # 各Foldでの検証データでの精度を格納
  y_pred = model.predict(va_x)
  val_scores.append(mean_squared_error(va_y, y_pred, squared=False))

train_score = np.mean(train_scores)
val_score = np.mean(val_scores)

In [6]:
train_score, val_score

(96.57332457973357, 137.7046239575256)